# Bulk RNA-seq Analysis with Single-Cell Integration

## Overview
This notebook integrates bulk RNA-seq data with single-cell RNA-seq data to create a comprehensive expression dataset. It processes single-cell data from both pre- and post-synaptic neurons, aggregates expression by sample, and merges with bulk RNA-seq data for comparative analysis.

## Objectives
1. Load and process single-cell RNA-seq data (iGlut_post and iGlut_pre)
2. Aggregate single-cell expression data by sample/parse_id
3. Merge single-cell aggregated data with bulk RNA-seq data  
4. Create marker gene expression heatmaps
5. Export integrated datasets for downstream analysis

## Expected Outputs
- Integrated expression matrix (bulk + single-cell)
- Integrated metadata file
- Marker gene expression heatmaps
- Sample-aggregated single-cell expression profiles

## Input Requirements
- Single-cell H5AD files (iGlut_post and iGlut_pre)
- Single-cell metadata files
- Bulk RNA-seq expression and metadata
- Marker gene lists for visualization

## Workflow Steps
1. **Data Loading**: Load single-cell and bulk datasets
2. **SC Processing**: Aggregate single-cell data by sample
3. **Data Integration**: Merge bulk and single-cell datasets
4. **Visualization**: Create expression heatmaps using marker genes
5. **Export**: Save integrated datasets for further analysis

## Configuration and Parameters

In [ ]:
# Centralized Configuration
import os
import warnings
warnings.filterwarnings('ignore')

# Base directories
BASE_DIR = os.getcwd()
BULK_DIR = os.path.join(BASE_DIR, "bulk")
SC_DATA_DIR = "/home/jjanssens/jjans/analysis/iNeuron_morphogens/final"
OUTPUT_DIR = os.path.join(BASE_DIR, "figures")

# Single-cell data paths
SC_SCANPY_DIR = os.path.join(SC_DATA_DIR, "scanpy")
SC_MARKERS_DIR = os.path.join(SC_DATA_DIR, "marker_genes")

# Input files
BULK_EXP_FILE = os.path.join(BULK_DIR, "reproducibility_genotypes.tsv")
BULK_META_FILE = os.path.join(BULK_DIR, "reproducibility_genotypes_meta.tsv")

# Output files
OUTPUT_EXP_FILE = os.path.join(BULK_DIR, "reproducibility_genotypes_wSC.tsv")
OUTPUT_META_FILE = os.path.join(BULK_DIR, "reproducibility_genotypes_wSC_meta.tsv")

# Sample configurations
SAMPLES_TO_PROCESS = ['iGlut_post', 'iGlut_pre']
SELECTED_PARSE_IDS = ['p1_D4', 'p1_D8', 'p1_D10', 'p1_B4', 'p1_B8', 'p1_B10',
                     'p3_C2', 'p3_F2', 'p3_D1', 'p3_F4', 'p3_G1', 'p3_G10']

# Sample ID mapping
SAMPLE_ID_MAPPING = {
    '1': 'p1_D4',   '2': 'p1_D8',   '3': 'p1_D10',
    '4': 'p1_B4',   '5': 'p1_B8',   '6': 'p1_B10',
    '7': 'p3_C2',   '8': 'p3_F2',   '9': 'p3_D1',
    '10': 'p3_F4',  '11': 'p3_G1',  '12': 'p3_G10'
}

# Analysis parameters
N_MARKER_GENES_PER_CLUSTER = 20
LOG_FOLD_CHANGE_THRESHOLD = 2.0
PADJ_THRESHOLD = 0.05
FIGURE_DPI = 350
HEATMAP_SIZE = (20, 20)

# Create output directories
os.makedirs(BULK_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "marker_genes"), exist_ok=True)

print("Configuration loaded successfully")
print(f"Base directory: {BASE_DIR}")
print(f"Single-cell data directory: {SC_DATA_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Samples to process: {SAMPLES_TO_PROCESS}")
print(f"Selected parse IDs: {len(SELECTED_PARSE_IDS)}")

## Library Imports and Helper Functions

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import sys
from pathlib import Path

# Single-cell analysis
import scanpy as sc
sc.settings.verbosity = 1  # Reduce scanpy verbosity

# Visualization
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# Set plotting parameters
plt.style.use('default')
sns.set_palette("husl")
mpl.rcParams['figure.dpi'] = 100

print("Libraries imported successfully")
print(f"Python version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Scanpy version: {sc.__version__}")
print(f"Matplotlib version: {mpl.__version__}")

# Helper Functions
def cluster_avg_expr(adata, clustering_key):
    """
    Calculate average expression per cluster.
    
    Parameters:
    -----------
    adata : AnnData
        Annotated data object
    clustering_key : str
        Column name in adata.obs containing cluster assignments
        
    Returns:
    --------
    pd.DataFrame
        Average expression matrix (genes x clusters)
    """
    if clustering_key not in adata.obs.columns:
        raise ValueError(f"Clustering key '{clustering_key}' not found in adata.obs")
    
    # Ensure clustering is categorical
    if not pd.api.types.is_categorical_dtype(adata.obs[clustering_key]):
        adata.obs[clustering_key] = adata.obs[clustering_key].astype('category')
    
    clusters = adata.obs[clustering_key].cat.categories
    cluster_avg = pd.DataFrame(
        columns=adata.var_names, 
        index=clusters,
        dtype=float
    )
    
    print(f"Calculating average expression for {len(clusters)} clusters...")
    
    for cluster in clusters:
        cluster_mask = adata.obs[clustering_key] == cluster
        n_cells = cluster_mask.sum()
        
        if n_cells > 0:
            # Sum expression across cells in cluster
            cluster_avg.loc[cluster] = adata[cluster_mask, :].X.sum(0).A1
        else:
            print(f"Warning: No cells found in cluster {cluster}")
            cluster_avg.loc[cluster] = 0
    
    # Transpose to genes x clusters format
    cluster_avg = cluster_avg.T.astype(float)
    
    print(f"Average expression matrix shape: {cluster_avg.shape}")
    return cluster_avg

def load_single_cell_data(sample_name):
    """
    Load single-cell data for a given sample.
    
    Parameters:
    -----------
    sample_name : str
        Name of the sample (e.g., 'iGlut_post', 'iGlut_pre')
        
    Returns:
    --------
    tuple
        (adata, metadata) - AnnData object and metadata DataFrame
    """
    print(f"Loading single-cell data for {sample_name}...")
    
    # Construct file paths
    h5ad_file = os.path.join(SC_SCANPY_DIR, f"{sample_name}_dr_clustered_raw_merged.h5ad")
    
    if sample_name == 'iGlut_pre':
        meta_file = os.path.join(SC_SCANPY_DIR, f"{sample_name}_dr_clustered_raw_merged_meta_fixed.tsv")
    else:
        meta_file = os.path.join(SC_SCANPY_DIR, f"{sample_name}_dr_clustered_raw_merged_meta.tsv")
    
    # Check file existence
    if not os.path.exists(h5ad_file):
        raise FileNotFoundError(f"H5AD file not found: {h5ad_file}")
    if not os.path.exists(meta_file):
        raise FileNotFoundError(f"Metadata file not found: {meta_file}")
    
    # Load data
    adata = sc.read_h5ad(h5ad_file)
    metadata = pd.read_csv(meta_file, sep="\t", index_col=0)
    
    print(f"Loaded {sample_name}: {adata.shape[0]} cells × {adata.shape[1]} genes")
    print(f"Metadata shape: {metadata.shape}")
    
    return adata, metadata

print("Helper functions defined successfully")



## Single-Cell Data Processing

In [ ]:
sample = 'iGlut_post'
adata1 = sc.read_h5ad("/home/jjanssens/jjans/analysis/iNeuron_morphogens/final/scanpy/"+sample+"_dr_clustered_raw_merged.h5ad")

sc_meta_post = pd.read_csv("/home/jjanssens/jjans/analysis/iNeuron_morphogens/final/scanpy/"+sample+"_dr_clustered_raw_merged_meta.tsv",sep="\t",index_col=0)
sc_meta_post.head()

def process_single_cell_sample(sample_name):
    """
    Process single-cell data for one sample and return aggregated expression.
    
    Parameters:
    -----------
    sample_name : str
        Sample name ('iGlut_post' or 'iGlut_pre')
        
    Returns:
    --------
    pd.DataFrame
        Aggregated expression by parse_id (genes x samples)
    """
    print(f"\n=== Processing {sample_name} ===")
    
    try:
        # Load data
        adata, sc_metadata = load_single_cell_data(sample_name)
        
        # Update adata.obs with metadata
        # Only update cells that exist in both datasets
        common_cells = adata.obs.index.intersection(sc_metadata.index)
        print(f"Common cells between adata and metadata: {len(common_cells)}")
        
        # Update adata.obs with metadata for common cells
        adata.obs = adata.obs.loc[common_cells]
        adata = adata[common_cells, :]
        adata.obs.update(sc_metadata.loc[common_cells])
        
        # Ensure parse_id is categorical
        if 'parse_id' not in adata.obs.columns:
            raise ValueError(f"'parse_id' column not found in {sample_name} metadata")
        
        adata.obs['parse_id'] = adata.obs['parse_id'].astype('category')
        
        # Calculate average expression per parse_id (sample)
        parse_id_expr = cluster_avg_expr(adata, 'parse_id')
        
        print(f"Parse ID expression shape: {parse_id_expr.shape}")
        print(f"Parse IDs found: {list(parse_id_expr.columns)}")
        
        # Fix metadata if needed (for pre-synaptic samples)
        if sample_name == 'iGlut_pre':
            # Save fixed metadata
            fixed_meta_file = os.path.join(SC_SCANPY_DIR, f"{sample_name}_dr_clustered_raw_merged_meta_fixed.tsv")
            sc_metadata.to_csv(fixed_meta_file, sep="\t")
            print(f"Fixed metadata saved to: {fixed_meta_file}")
        
        return parse_id_expr
        
    except Exception as e:
        print(f"Error processing {sample_name}: {e}")
        return None

# Process both single-cell samples
sc_expression_data = {}

for sample in SAMPLES_TO_PROCESS:
    expr_data = process_single_cell_sample(sample)
    if expr_data is not None:
        sc_expression_data[sample] = expr_data
    else:
        print(f"Failed to process {sample}")

print(f"\nSuccessfully processed {len(sc_expression_data)} single-cell samples")
for sample, data in sc_expression_data.items():
    print(f"  {sample}: {data.shape[0]} genes × {data.shape[1]} samples")

,sample,species,gene_count,tscp_count,mread_count,bc1_well,bc2_well,bc3_well,bc1_wind,bc2_wind,...,M_CHIR,M_RA,M_FGF8,M_BMP4,M_SHH,M_PM,tSNE_1,tSNE_2,umap_1,umap_2
01_01_02__s1,iGlut_post_p1,hg38,3542,9141,11365,A1,A1,A2,1,1,...,0.0,0,0,50,0,0,0.542512,39.629030,-1.988116,7.912624
01_01_12__s1,iGlut_post_p1,hg38,1176,1940,2390,A1,A1,A12,1,1,...,0.0,0,0,50,0,0,6.332805,39.521160,-3.288275,8.063947
01_01_16__s1,iGlut_post_p1,hg38,3393,8576,10612,A1,A1,B4,1,1,...,0.0,0,0,50,0,0,28.295320,26.426134,-1.621965,-1.320243
01_01_22__s1,iGlut_post_p1,hg38,2650,5700,7061,A1,A1,B10,1,1,...,0.0,0,0,50,0,0,42.816700,21.352268,-1.820428,-2.448944
01_01_55__s1,iGlut_post_p1,hg38,1970,3873,4844,A1,A1,E7,1,1,...,0.0,0,0,50,0,0,19.640999,-2.478396,3.294251,-1.841123


In [ ]:
def merge_single_cell_data():
    """
    Merge single-cell expression data from post and pre samples.
    
    Returns:
    --------
    pd.DataFrame
        Combined expression matrix (genes x all_samples)
    """
    print("\n=== Merging Single-Cell Data ===")
    
    if len(sc_expression_data) != 2:
        print(f"Warning: Expected 2 samples, got {len(sc_expression_data)}")
        return None
    
    # Get expression data
    expr_post = sc_expression_data.get('iGlut_post')
    expr_pre = sc_expression_data.get('iGlut_pre')
    
    if expr_post is None or expr_pre is None:
        print("Error: Missing expression data for post or pre samples")
        return None
    
    # Merge on gene index
    print(f"Merging expression data:")
    print(f"  Post-synaptic: {expr_post.shape}")
    print(f"  Pre-synaptic: {expr_pre.shape}")
    
    # Find common genes
    common_genes = expr_post.index.intersection(expr_pre.index)
    print(f"  Common genes: {len(common_genes)}")
    
    # Merge data
    merged_expr = pd.merge(
        expr_post.loc[common_genes], 
        expr_pre.loc[common_genes], 
        left_index=True, 
        right_index=True
    )
    
    print(f"Merged single-cell expression: {merged_expr.shape}")
    return merged_expr

def create_sc_metadata():
    """
    Create metadata for single-cell aggregated samples.
    
    Returns:
    --------
    pd.DataFrame
        Metadata for single-cell samples
    """
    print("Creating single-cell metadata...")
    
    # Create metadata DataFrame
    sc_meta = pd.DataFrame(index=SELECTED_PARSE_IDS)
    sc_meta['genotype'] = 'single_cell'
    sc_meta['repl'] = '1'
    
    # Add sample mapping
    for sample_id, parse_id in SAMPLE_ID_MAPPING.items():
        if parse_id in sc_meta.index:
            sc_meta.loc[parse_id, 'sample'] = sample_id
    
    print(f"Single-cell metadata created: {sc_meta.shape}")
    return sc_meta

# Merge single-cell data
parse_id_sum_expr_all = merge_single_cell_data()

if parse_id_sum_expr_all is not None:
    # Select only the samples we want
    available_samples = [s for s in SELECTED_PARSE_IDS if s in parse_id_sum_expr_all.columns]
    parse_id_sum_expr_all_sel = parse_id_sum_expr_all[available_samples].copy()
    
    print(f"Selected samples: {len(available_samples)}")
    print(f"Final single-cell expression shape: {parse_id_sum_expr_all_sel.shape}")
    
    # Create metadata
    sc_metadata = create_sc_metadata()
    
    # Ensure metadata matches expression data
    common_samples = parse_id_sum_expr_all_sel.columns.intersection(sc_metadata.index)
    parse_id_sum_expr_all_sel = parse_id_sum_expr_all_sel[common_samples]
    sc_metadata = sc_metadata.loc[common_samples]
    
    print(f"Final aligned data: {parse_id_sum_expr_all_sel.shape[1]} samples")
else:
    print("Failed to merge single-cell data")

## Bulk Data Integration

In [ ]:
def load_bulk_data():
    """
    Load bulk RNA-seq expression and metadata.
    
    Returns:
    --------
    tuple
        (expression_df, metadata_df) - Bulk expression and metadata DataFrames
    """
    print("Loading bulk RNA-seq data...")
    
    try:
        # Load bulk expression data
        bulk_exp = pd.read_csv(BULK_EXP_FILE, sep="\t", index_col=0)
        print(f"Bulk expression loaded: {bulk_exp.shape}")
        
        # Load bulk metadata
        bulk_meta = pd.read_csv(BULK_META_FILE, sep="\t", index_col=0)
        print(f"Bulk metadata loaded: {bulk_meta.shape}")
        
        return bulk_exp, bulk_meta
        
    except FileNotFoundError as e:
        print(f"Error: Bulk data file not found - {e}")
        return None, None
    except Exception as e:
        print(f"Error loading bulk data: {e}")
        return None, None

def integrate_bulk_and_sc_data(bulk_exp, bulk_meta, sc_exp, sc_meta):
    """
    Integrate bulk and single-cell data into combined datasets.
    
    Parameters:
    -----------
    bulk_exp : pd.DataFrame
        Bulk expression data
    bulk_meta : pd.DataFrame
        Bulk metadata
    sc_exp : pd.DataFrame
        Single-cell aggregated expression
    sc_meta : pd.DataFrame
        Single-cell metadata
        
    Returns:
    --------
    tuple
        (merged_exp, merged_meta) - Integrated expression and metadata
    """
    print("\n=== Integrating Bulk and Single-Cell Data ===")
    
    # Check data validity
    if any(data is None for data in [bulk_exp, bulk_meta, sc_exp, sc_meta]):
        print("Error: Missing required data for integration")
        return None, None
    
    print(f"Input data shapes:")
    print(f"  Bulk expression: {bulk_exp.shape}")
    print(f"  Bulk metadata: {bulk_meta.shape}")
    print(f"  SC expression: {sc_exp.shape}")
    print(f"  SC metadata: {sc_meta.shape}")
    
    # Find common genes
    common_genes = bulk_exp.index.intersection(sc_exp.index)
    print(f"Common genes: {len(common_genes)}")
    
    if len(common_genes) == 0:
        print("Error: No common genes between bulk and single-cell data")
        return None, None
    
    # Merge expression data
    merged_exp = pd.merge(
        bulk_exp.loc[common_genes], 
        sc_exp.loc[common_genes], 
        left_index=True, 
        right_index=True
    )
    
    # Merge metadata
    merged_meta = pd.concat([bulk_meta, sc_meta], axis=0)
    
    print(f"Integrated data:")
    print(f"  Expression: {merged_exp.shape}")
    print(f"  Metadata: {merged_meta.shape}")
    
    # Validate consistency
    if set(merged_exp.columns) != set(merged_meta.index):
        print("Warning: Sample mismatch between expression and metadata")
        common_samples = merged_exp.columns.intersection(merged_meta.index)
        merged_exp = merged_exp[common_samples]
        merged_meta = merged_meta.loc[common_samples]
        print(f"After alignment: Expression {merged_exp.shape}, Metadata {merged_meta.shape}")
    
    return merged_exp, merged_meta

# Load bulk data
bulk_expression, bulk_metadata = load_bulk_data()

# Integrate data if both bulk and SC data are available
if (bulk_expression is not None and bulk_metadata is not None and 
    parse_id_sum_expr_all_sel is not None and sc_metadata is not None):
    
    merged_expression, merged_metadata = integrate_bulk_and_sc_data(
        bulk_expression, bulk_metadata, 
        parse_id_sum_expr_all_sel, sc_metadata
    )
    
    if merged_expression is not None and merged_metadata is not None:
        print(f"\n✓ Data integration successful!")
        print(f"Final integrated dataset: {merged_expression.shape[0]} genes × {merged_expression.shape[1]} samples")
    else:
        print("✗ Data integration failed")
else:
    print("✗ Cannot integrate data - missing required datasets")
    merged_expression, merged_metadata = None, None

In [ ]:
def export_integrated_data(expression_data, metadata, exp_file, meta_file):
    """
    Export integrated expression and metadata to files.
    
    Parameters:
    -----------
    expression_data : pd.DataFrame
        Integrated expression data
    metadata : pd.DataFrame
        Integrated metadata
    exp_file : str
        Output file path for expression data
    meta_file : str
        Output file path for metadata
    """
    print("\n=== Exporting Integrated Data ===")
    
    if expression_data is None or metadata is None:
        print("Error: Cannot export - missing data")
        return False
    
    try:
        # Export expression data
        expression_data.to_csv(exp_file, sep="\t")
        print(f"Expression data exported: {exp_file}")
        print(f"  Shape: {expression_data.shape}")
        
        # Export metadata
        metadata.to_csv(meta_file, sep="\t")
        print(f"Metadata exported: {meta_file}")
        print(f"  Shape: {metadata.shape}")
        
        # Validate exported files
        if os.path.exists(exp_file) and os.path.exists(meta_file):
            exp_size = os.path.getsize(exp_file) / (1024*1024)  # MB
            meta_size = os.path.getsize(meta_file) / (1024*1024)  # MB
            print(f"File sizes: Expression {exp_size:.1f}MB, Metadata {meta_size:.1f}MB")
            return True
        else:
            print("Error: Files were not created successfully")
            return False
            
    except Exception as e:
        print(f"Error exporting data: {e}")
        return False

# Export integrated data
if merged_expression is not None and merged_metadata is not None:
    export_success = export_integrated_data(
        merged_expression, merged_metadata,
        OUTPUT_EXP_FILE, OUTPUT_META_FILE
    )
    
    if export_success:
        print("\n✓ Data export completed successfully!")
        
        # Display summary
        print(f"\nDataset Summary:")
        print(f"  Total samples: {merged_expression.shape[1]}")
        print(f"  Bulk samples: {bulk_expression.shape[1] if bulk_expression is not None else 0}")
        print(f"  Single-cell samples: {parse_id_sum_expr_all_sel.shape[1] if parse_id_sum_expr_all_sel is not None else 0}")
        print(f"  Total genes: {merged_expression.shape[0]}")
        
        print(f"\nGenotype distribution:")
        if 'genotype' in merged_metadata.columns:
            genotype_counts = merged_metadata['genotype'].value_counts()
            for genotype, count in genotype_counts.items():
                print(f"  {genotype}: {count} samples")
    else:
        print("✗ Data export failed")
else:
    print("✗ No integrated data to export")

## Marker Gene Visualization

In [ ]:
def create_cluster_expression_analysis():
    """
    Create cluster-level expression analysis and visualizations.
    This function only runs if single-cell data processing was successful.
    """
    print("\n=== Cluster Expression Analysis ===")
    
    # Check if we have the necessary single-cell data
    if 'iGlut_pre' not in sc_expression_data:
        print("Single-cell pre-synaptic data not available - skipping cluster analysis")
        return None
    
    try:
        # Reload the pre-synaptic data for cluster analysis
        sample_name = 'iGlut_pre'
        adata, sc_metadata = load_single_cell_data(sample_name)
        
        # Update adata.obs with metadata
        common_cells = adata.obs.index.intersection(sc_metadata.index)
        adata = adata[common_cells, :]
        adata.obs.update(sc_metadata.loc[common_cells])
        
        # Ensure clustering columns are categorical
        clustering_columns = ['merged_clusters_from_10', 'final_clustering']
        available_columns = [col for col in clustering_columns if col in adata.obs.columns]
        
        if not available_columns:
            print("No clustering columns found - skipping cluster analysis")
            return None
            
        print(f"Available clustering columns: {available_columns}")
        
        # Use final_clustering if available, otherwise use merged_clusters_from_10
        cluster_col = 'final_clustering' if 'final_clustering' in available_columns else available_columns[0]
        
        # Ensure clustering is categorical
        adata.obs[cluster_col] = adata.obs[cluster_col].astype('category')
        
        # Calculate cluster average expression
        cluster_avg = cluster_avg_expr(adata, cluster_col)
        
        # Calculate CPM and log-CPM
        cluster_avg_CPM = cluster_avg / cluster_avg.sum() * 1e6
        cluster_avg_logCPM = np.log1p(cluster_avg_CPM)
        
        print(f"Cluster expression calculated: {cluster_avg.shape}")
        print(f"Available clusters: {list(cluster_avg.columns)}")
        
        return {
            'cluster_avg': cluster_avg,
            'cluster_avg_CPM': cluster_avg_CPM, 
            'cluster_avg_logCPM': cluster_avg_logCPM,
            'clustering_used': cluster_col
        }
        
    except Exception as e:
        print(f"Error in cluster expression analysis: {e}")
        return None

def load_and_process_marker_genes():
    """
    Load marker genes and prepare them for visualization.
    """
    print("Loading marker genes...")
    
    try:
        marker_file = os.path.join(SC_MARKERS_DIR, "iGlut_pre_dr_clustered_raw_merged_markers.tsv")
        
        if not os.path.exists(marker_file):
            print(f"Marker file not found: {marker_file}")
            return None
            
        marker_genes = pd.read_csv(marker_file, sep="\t", index_col=0)
        print(f"Marker genes loaded: {marker_genes.shape}")
        
        # Filter for positive markers with significance
        marker_genes_pos = marker_genes.loc[marker_genes['logfoldchanges'] > 0].copy()
        marker_genes_pos = marker_genes_pos.loc[marker_genes_pos['pvals_adj'] < PADJ_THRESHOLD]
        marker_genes_pos_sig = marker_genes_pos.loc[marker_genes_pos['logfoldchanges'] > LOG_FOLD_CHANGE_THRESHOLD]
        
        print(f"Positive markers: {len(marker_genes_pos)}")
        print(f"Significant markers (FC > {LOG_FOLD_CHANGE_THRESHOLD}): {len(marker_genes_pos_sig)}")
        
        # Select top markers per cluster
        genes_for_heatmap = list(marker_genes_pos.groupby('cluster').head(n=N_MARKER_GENES_PER_CLUSTER).index)
        genes_for_heatmap_sig = list(marker_genes_pos_sig.groupby('cluster').head(n=N_MARKER_GENES_PER_CLUSTER).index)
        
        print(f"Selected markers for heatmap: {len(genes_for_heatmap)}")
        print(f"Selected significant markers: {len(genes_for_heatmap_sig)}")
        
        return {
            'all_markers': marker_genes,
            'positive_markers': marker_genes_pos,
            'significant_markers': marker_genes_pos_sig,
            'heatmap_genes': genes_for_heatmap,
            'heatmap_genes_sig': genes_for_heatmap_sig
        }
        
    except Exception as e:
        print(f"Error loading marker genes: {e}")
        return None

# Perform cluster expression analysis
cluster_results = create_cluster_expression_analysis()

# Load marker genes
marker_results = load_and_process_marker_genes()

if cluster_results is not None and marker_results is not None:
    print("\n✓ Cluster analysis and marker gene loading completed successfully!")
else:
    if cluster_results is None:
        print("⚠ Cluster analysis not available")
    if marker_results is None:
        print("⚠ Marker gene analysis not available")

In [ ]:
marker_genes_pos

def create_marker_heatmaps():
    """
    Create heatmaps for marker gene expression across clusters.
    """
    if cluster_results is None or marker_results is None:
        print("Cannot create heatmaps - missing cluster or marker data")
        return
        
    print("\n=== Creating Marker Gene Heatmaps ===")
    
    try:
        cluster_avg_logCPM = cluster_results['cluster_avg_logCPM']
        genes_for_heatmap_sig = marker_results['heatmap_genes_sig']
        
        # Filter genes that exist in expression data
        available_genes = [g for g in genes_for_heatmap_sig if g in cluster_avg_logCPM.index]
        
        if len(available_genes) == 0:
            print("No marker genes available in expression data")
            return
            
        print(f"Creating heatmap with {len(available_genes)} marker genes")
        
        # Prepare expression data for heatmap
        expr_plot = cluster_avg_logCPM.loc[available_genes].copy()
        expr_plot_Z = (expr_plot.T - expr_plot.T.mean()) / expr_plot.T.std()
        expr_plot_Z = expr_plot_Z.dropna(axis=1)
        
        # Create heatmap
        plt.figure(figsize=HEATMAP_SIZE)
        sns.heatmap(expr_plot_Z.T, vmax=2, vmin=0, cmap='viridis', 
                   cbar_kws={'label': 'Z-score'})
        plt.title(f'Marker Gene Expression - {cluster_results["clustering_used"]}', 
                 fontsize=16, fontweight='bold')
        plt.xlabel('Clusters', fontsize=12)
        plt.ylabel('Marker Genes', fontsize=12)
        plt.tight_layout()
        
        # Save plot
        save_path = os.path.join(OUTPUT_DIR, "marker_genes", "iGlut_pre_final_clustering_heatmap_sig.png")
        plt.savefig(save_path, dpi=FIGURE_DPI, bbox_inches='tight', pad_inches=0.1)
        print(f"Heatmap saved: {save_path}")
        
        plt.show()
        plt.close()
        
        return expr_plot_Z
        
    except Exception as e:
        print(f"Error creating heatmaps: {e}")
        return None

def create_analysis_summary():
    """
    Create comprehensive analysis summary.
    """
    print("\n" + "="*60)
    print("BULK + SINGLE-CELL INTEGRATION ANALYSIS - SUMMARY")
    print("="*60)
    
    print(f"\n📊 DATA PROCESSING SUMMARY:")
    if len(sc_expression_data) > 0:
        for sample, data in sc_expression_data.items():
            print(f"  • {sample}: {data.shape[0]} genes × {data.shape[1]} samples")
    else:
        print("  • No single-cell data processed")
    
    if bulk_expression is not None:
        print(f"  • Bulk data: {bulk_expression.shape[0]} genes × {bulk_expression.shape[1]} samples")
    else:
        print("  • Bulk data: Not loaded")
    
    if merged_expression is not None:
        print(f"  • Integrated: {merged_expression.shape[0]} genes × {merged_expression.shape[1]} samples")
    else:
        print("  • Integration: Failed")
    
    print(f"\n🔬 ANALYSIS RESULTS:")
    print(f"  • Single-cell samples processed: {len(SAMPLES_TO_PROCESS)}")
    print(f"  • Selected parse IDs: {len(SELECTED_PARSE_IDS)}")
    
    if cluster_results is not None:
        print(f"  • Cluster analysis: ✓ Completed ({cluster_results['clustering_used']})")
        print(f"  • Clusters found: {cluster_results['cluster_avg'].shape[1]}")
    else:
        print(f"  • Cluster analysis: ✗ Not completed")
    
    if marker_results is not None:
        print(f"  • Marker genes: ✓ Loaded ({len(marker_results['significant_markers'])} significant)")
    else:
        print(f"  • Marker genes: ✗ Not loaded")
    
    print(f"\n📁 OUTPUT FILES:")
    output_files = [OUTPUT_EXP_FILE, OUTPUT_META_FILE]
    for filepath in output_files:
        if os.path.exists(filepath):
            size_mb = os.path.getsize(filepath) / (1024*1024)
            print(f"  ✓ {os.path.basename(filepath)} ({size_mb:.1f}MB)")
        else:
            print(f"  ✗ {os.path.basename(filepath)} (not created)")
    
    print(f"\n⚙️  CONFIGURATION USED:")
    print(f"  • Samples processed: {SAMPLES_TO_PROCESS}")
    print(f"  • Markers per cluster: {N_MARKER_GENES_PER_CLUSTER}")
    print(f"  • Log FC threshold: {LOG_FOLD_CHANGE_THRESHOLD}")
    print(f"  • P-adj threshold: {PADJ_THRESHOLD}")
    
    print("\n" + "="*60)

# Create visualizations if data is available
heatmap_result = create_marker_heatmaps()

# Create final summary
create_analysis_summary()

print("\n✅ Bulk + Single-Cell Integration Analysis Completed!")
print(f"All outputs saved to: {OUTPUT_DIR}")

,scores,pvals,pvals_adj,logfoldchanges,cluster,fcluster,cluster_old
BOC,69.443950,0.000000,0.000000,2.953943,0,0,0
MIR99AHG,60.817980,0.000000,0.000000,2.048046,0,0,0
HOXD3,60.788260,0.000000,0.000000,4.283298,0,0,0
RFX4,54.793470,0.000000,0.000000,2.647338,0,0,0
PRTG,54.129130,0.000000,0.000000,1.525106,0,0,0
...,...,...,...,...,...,...,...
MAGED1,2.978668,0.002895,0.048924,0.117719,114,114,114
NETO2,2.975626,0.002924,0.049370,0.132684,114,114,114
DTNBP1,2.974293,0.002937,0.049558,0.225376,114,114,114
SYS1-DBNDD2,2.972491,0.002954,0.049836,0.768933,114,114,114


In [ ]:
## Session Information

['BOC',
 'MIR99AHG',
 'HOXD3',
 'RFX4',
 'PRTG',
 'RHBDL3',
 'LINC03000',
 'PDE1A',
 'MLLT3',
 'PAX3',
 'DCC',
 'ENSG00000259692',
 'PDE4B',
 'CSRNP3',
 'ASXL3',
 'GRID2',
 'PVT1',
 'LRIG1',
 'PCLO',
 'LINC01473',
 'RMST',
 'CDH6',
 'ADAMTS18',
 'TRPM3',
 'WLS',
 'TMEM132D',
 'ERBB4',
 'SLIT2',
 'EFNA5',
 'GLIS3',
 'PCDH7',
 'NR2F1-AS1',
 'SHROOM3',
 'MMRN1',
 'MIR99AHG',
 'COL4A6',
 'FBN2',
 'NEBL',
 'ADGRL3',
 'KIAA1217',
 'SDK1',
 'PRKG1',
 'TALAM1',
 'THSD7B',
 'NAV2',
 'NRXN1',
 'CSMD1',
 'CNTNAP2',
 'SEMA6D',
 'KCNIP4',
 'DCC',
 'AKAP6',
 'NALF1',
 'CCSER1',
 'EBF1',
 'EGFEM1P',
 'CADM1',
 'THSD7A',
 'RBMS3',
 'CTNNA2',
 'EFNA5',
 'TRPM3',
 'PCDH7',
 'ADAMTS18',
 'KIAA1217',
 'RMST',
 'SLIT2',
 'GLIS3',
 'LAMB1',
 'CADPS2',
 'ADGRL3',
 'FBN2',
 'GPC6',
 'PCSK5',
 'B3GALT1',
 'NEBL',
 'LTBP1',
 'LRP2',
 'TIAM1',
 'PVT1',
 'LINC01414',
 'BOC',
 'PAX3',
 'NPAS3',
 'MIR99AHG',
 'FGF13',
 'LRIG1',
 'LINC03000',
 'LINC02306',
 'PAX7',
 'GRID2',
 'RFX4',
 'EGFEM1P',
 'ERBB4',
 'KIF21A',

In [ ]:
# Session Information for Reproducibility
import datetime
import platform

print("🔧 SESSION INFORMATION")
print("-" * 40)
print(f"Analysis completed: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Platform: {platform.system()} {platform.release()}")
print(f"Python version: {sys.version}")
print(f"Working directory: {os.getcwd()}")

print(f"\n📦 PACKAGE VERSIONS:")
packages = ['pandas', 'numpy', 'scanpy', 'matplotlib', 'seaborn']
for package in packages:
    try:
        if package == 'scanpy':
            print(f"  • {package}: {sc.__version__}")
        else:
            module = __import__(package)
            version = getattr(module, '__version__', 'Unknown')
            print(f"  • {package}: {version}")
    except ImportError:
        print(f"  • {package}: Not installed")

print(f"\n🔄 ANALYSIS REPRODUCIBILITY:")
print(f"  • Configuration: Centralized parameters")
print(f"  • Data paths: Parameterized")
print(f"  • Random seed: Not applicable (deterministic)")
print(f"  • Output files: Saved with timestamps")

print(f"\n📊 FINAL DATASET SUMMARY:")
if merged_expression is not None:
    print(f"  • Integrated expression data: {merged_expression.shape}")
    print(f"  • Sample types in metadata:")
    if merged_metadata is not None and 'genotype' in merged_metadata.columns:
        for genotype, count in merged_metadata['genotype'].value_counts().items():
            print(f"    - {genotype}: {count} samples")

print("\n✅ Bulk + Single-Cell Integration Analysis completed successfully!")
print("All results exported and analysis documented.")

In [46]:
len(set(merged_exp.index))

60344

In [47]:
len(merged_exp.index)

60344

In [ ]:
parse_id_sum_expr_all

In [ ]:
	genotype	repl	sample	sums
	<chr>	<int>	<int>	<dbl>
Reproducibility_SC102A1_1_4	SC102A1	1	4	1040799
Reproducibility_SC102A1_1_9	SC102A1	1	9	2918046
Reproducibility_SC102A1_3_7	SC102A1	3	7	3436371
Reproducibility_SC102A1_2_12	SC102A1	2	12	695566
Reproducibility_SC102A1_3_8	SC102A1	3	

In [19]:
parse_id_sum_expr_pre

,p3_A1,p3_A10,p3_A11,p3_A12,p3_A2,p3_A3,p3_A4,p3_A5,p3_A6,p3_A7,...,p3_H11,p3_H12,p3_H2,p3_H3,p3_H4,p3_H5,p3_H6,p3_H7,p3_H8,p3_H9
TSPAN6,28.0,258.0,296.0,75.0,22.0,38.0,75.0,39.0,30.0,43.0,...,162.0,202.0,119.0,132.0,139.0,149.0,108.0,133.0,151.0,331.0
TNMD,4.0,0.0,1.0,1.0,1.0,17.0,7.0,0.0,5.0,1.0,...,2.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,2.0
DPM1,163.0,755.0,831.0,267.0,194.0,278.0,369.0,205.0,276.0,327.0,...,503.0,651.0,328.0,406.0,307.0,365.0,340.0,384.0,349.0,877.0
SCYL3,125.0,381.0,436.0,195.0,151.0,176.0,249.0,149.0,214.0,197.0,...,183.0,355.0,233.0,155.0,156.0,191.0,152.0,152.0,165.0,501.0
C1orf112,169.0,1495.0,1915.0,318.0,159.0,247.0,327.0,176.0,204.0,224.0,...,656.0,1197.0,731.0,541.0,486.0,457.0,514.0,526.0,582.0,1526.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000291313,1.0,1.0,3.0,0.0,2.0,0.0,1.0,2.0,1.0,0.0,...,2.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,5.0
ENSG00000291314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000291315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000291316,2.0,6.0,3.0,7.0,3.0,13.0,4.0,7.0,13.0,5.0,...,9.0,7.0,5.0,4.0,3.0,8.0,3.0,9.0,11.0,7.0
